<a href="https://colab.research.google.com/github/travislatchman/Cardiac-Ultrasound-Image-Segmentation-and-Stroke-Volume-estimation/blob/main/Cardiac_Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import Reconstruction as Rec
import Segmentation as Seg

TwoSequence, FourSequence, aspectRatio = Seg.getSequenceSegmentations(1, False)

newTwo = Rec.resampleAllContours(TwoSequence)
newFour = Rec.resampleAllContours(FourSequence)

interpTwo = Rec.interpolateSequence(newTwo, 60)
interpFour = Rec.interpolateSequence(newFour, 60)

frames = []
for i in range(len(interpTwo)):
    frames.append(Rec.generatePointCloud(interpTwo[i], interpFour[i], 3, aspectRatio, i == 0))

**Interactive Widget**

In [2]:
import pyvista as pv
import numpy as np

p = pv.wrap(np.array(frames[10]))
surf = p.reconstruct_surface()

pl = pv.Plotter(shape=(1, 2))
pl.add_mesh(p, style='points', point_size=3.0)
pl.add_title('Point Cloud of 3D Surface')
pl.subplot(0, 1)
pl.add_mesh(surf, opacity = 0.4, show_edges=True)
pl.add_title('Reconstructed Surface')
pl.show()

Widget(value="<iframe src='http://localhost:58376/index.html?ui=P_0x145c8376c08_0&reconnect=auto' style='width…

**Create Static Movie**

In [3]:
import numpy as np
import pyvista as pv

plotter = pv.Plotter(shape=(1, 2))
plotter.open_movie("staticMovie.mp4")

reps = 10

for i in range(reps):
    for frame in frames:
        plotter.clear()
        
        p = pv.wrap(np.array(frame))
        surf = p.reconstruct_surface()

        plotter.subplot(0, 0)
        plotter.add_mesh(p, style='points', point_size=3.0)
        plotter.add_title('Point Cloud')
        plotter.subplot(0, 1)
        plotter.add_mesh(surf, opacity = 0.4, show_edges=True)
        plotter.add_title('Surface')
        
        plotter.write_frame()  # Write this frame
        
plotter.close()

**Create Moving Movie**

In [4]:
import numpy as np
import pyvista as pv

def lerp(start, end, t):
    return start * (1-t) + end*t

plotter = pv.Plotter(shape=(1, 2))
plotter.open_movie("movingMovie.mp4")

radius = 1200
reps = 10

totalFrames = reps * len(frames)
counter = 0
for i in range(reps):
    for frame in frames:
        plotter.clear()
        
        frame = np.array(frame)
        frame = (np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0]]) @ frame.T).T #Rotate coord system
        
        p = pv.wrap(frame)
        surf = p.reconstruct_surface()

        centerOfMass = np.average(frame, 0)
        phi = lerp(0, np.pi * 1.1, counter / totalFrames)
        theta = lerp(np.pi * 0.3, np.pi * 0.7, counter / totalFrames)
        pos = (radius * np.sin(theta) * np.cos(phi) + centerOfMass[0], radius * np.sin(theta) * np.sin(phi) + centerOfMass[1], radius * np.cos(theta) + centerOfMass[2])
        
        plotter.subplot(0, 0)
        plotter.add_mesh(p, style='points', point_size=3.0)
        plotter.add_title('Point Cloud')
        plotter.camera.position = pos
        plotter.subplot(0, 1)
        plotter.add_mesh(surf, opacity = 0.4, show_edges=True)
        plotter.add_title('Surface')
        
        plotter.camera.position = pos
        plotter.write_frame()  # Write this frame
        
        counter += 1
        
plotter.close()